### Разделил все файлы на обучающую и тестовую выборки. Отдельно для валидации перед этим случайно руками выбрал два десятка изображений

In [63]:
import os
from sklearn.model_selection import train_test_split
import shutil

In [47]:
files = sorted(os.listdir('11/train'))

In [48]:
files = list(x[:-4] for x in files)

In [49]:
len(files)

1524

In [50]:
files

['3-avi-frame417_jpg.rf.2e4440e1272a5ac19f1488623f459e68',
 '3-avi-frame417_jpg.rf.2e4440e1272a5ac19f1488623f459e68',
 '3-avi-frame418_jpg.rf.c31c6b58ea806f930a1854f3fcc224c5',
 '3-avi-frame418_jpg.rf.c31c6b58ea806f930a1854f3fcc224c5',
 '3-avi-frame420_jpg.rf.0a1295890e05add1f9208ea11cffce14',
 '3-avi-frame420_jpg.rf.0a1295890e05add1f9208ea11cffce14',
 '3-avi-frame420_jpg.rf.3622afd0ef7262bf190a80f5286f64bd',
 '3-avi-frame420_jpg.rf.3622afd0ef7262bf190a80f5286f64bd',
 '3-avi-frame421_jpg.rf.b29aedff10e784a7eb521443c9cc4ecd',
 '3-avi-frame421_jpg.rf.b29aedff10e784a7eb521443c9cc4ecd',
 '3-avi-frame422_jpg.rf.7e096020b42bfa17721ca510d5ae094e',
 '3-avi-frame422_jpg.rf.7e096020b42bfa17721ca510d5ae094e',
 '3-avi-frame423_jpg.rf.470148994b4410b5a7f7f7eb926b64e6',
 '3-avi-frame423_jpg.rf.470148994b4410b5a7f7f7eb926b64e6',
 '3-avi-frame424_jpg.rf.9aadddfacc9b48891f5414761b910e9f',
 '3-avi-frame424_jpg.rf.9aadddfacc9b48891f5414761b910e9f',
 '3-avi-frame424_jpg.rf.a192ecb1b1d327249fe1643e2dd4e3f8

In [51]:
len(set(files))

762

In [60]:
tr, val = train_test_split(list(set(files)), shuffle=True, test_size=0.15)

In [61]:
len(tr)

647

In [62]:
len(val)

115

In [64]:
for f in tr:
    shutil.copy(f'11/train/{f}.jpg', f'dataset/images/train/{f}.jpg')
    shutil.copy(f'11/train/{f}.txt', f'dataset/labels/train/{f}.txt')

In [65]:
for f in val:
    shutil.copy(f'11/train/{f}.jpg', f'dataset/images/val/{f}.jpg')
    shutil.copy(f'11/train/{f}.txt', f'dataset/labels/val/{f}.txt')

In [67]:
with open('classes.txt') as f: cls = f.read()

In [69]:
cls=cls.split('\n')

In [71]:
len(cls)

60

In [72]:
str(cls)

"['28', '29', '4', '5', '1', '8', '2', '37', '20', '41', '68', '26', '59', '14', '15', '49', '42', '43', '35', '27', '46', '12', '7', '13', '30', '36', '19', '47', '11', '6', '45', '44', '52', '57', '25', '60', '38', '51', '55', '39', '24', '23', '53', '31', '48', '61', '64', '66', '21', '10', '9', '16', '33', '63', '54', '22', '3', '17', '116', '']"

# Инференс на тестовых данных

In [86]:
import pandas as pd
import subprocess
import time

In [97]:
test_df = pd.read_csv('test.csv')
test_df

,id,img
0,754,6-avi-frame14887_jpg.rf.bb0bf6b4b122c23e1b33a9...
1,29,5-avi-frame2916_jpg.rf.1ecdbbc129d33896fd25b9b...
2,1157,5-avi-frame2871_jpg.rf.f73998176f8a19ee03f8704...
3,1049,6-avi-frame5752_jpg.rf.a067b0fc55b770c9b10bb7a...
4,715,6-avi-frame5678_jpg.rf.f140419d224703d49fe65db...
...,...,...
383,243,3-avi-frame433_jpg.rf.b29d8fb61048b3016805a62e...
384,921,6-avi-frame4125_jpg.rf.e1dbdc7a7421bc9bc95a58f...
385,881,6-avi-frame1424_jpg.rf.e171549738da66b200b537c...
386,503,5-avi-frame2867_jpg.rf.c0bbb8942eeb6fac582e6b6...


In [149]:
test_df[test_df.id==754].img.item()

'6-avi-frame14887_jpg.rf.bb0bf6b4b122c23e1b33a9e05d006b06.jpg'

In [161]:
submission = pd.DataFrame(columns=['id','sing1','sing2','sing3','sing4','sing5','sing6','sing7','sing8'])
submission
test_dir = '/home/andrey/Downloads/PyCharm_Projects/Krasnodar/test/'
i=0
for idts in list(test_df.id):
    fname = test_df[test_df.id==idts].img.item()
    cmd_line = f'cd /home/andrey/YOLOv6 && \
python3 /home/andrey/YOLOv6/tools/infer.py \
--weights /home/andrey/YOLOv6/runs/train/exp9/weights/best_ckpt.pt \
--source {test_dir}{fname} \
--device 0 \
--iou-thres 0.6 \
--conf-thres=0.5 \
--save-txt \
--save-dir /home/andrey/Downloads/PyCharm_Projects/Krasnodar/test_result/'
    # print(cmd_line)
    subprocess.run(cmd_line, shell=True)
    # time.sleep(1)
    if os.path.exists('/home/andrey/Downloads/PyCharm_Projects/Krasnodar/test_result/'+fname[:-3]+'txt'):
        with open('/home/andrey/Downloads/PyCharm_Projects/Krasnodar/test_result/'+fname[:-3]+'txt') as f:
            result = f.read();
    else: result = '';
    rst = [];
    #print(result);
    for line in result.split('\n'):
        if len(line) < 1: continue;
        ll = line.split()[-1].split('>>')
        rst.append((ll[0], ll[1]))
    while len(rst) > 8:
        rst.pop(0)
    while len(rst) < 8:
        rst.append(('0','0'))
    # print(str(rst))
    res = list(x[1] for x in sorted(rst, reverse=True))
    # idx = test_df[test_df['img'] == fname].id.item()
    res.insert(0, idts)
    submission.loc[i] = res
    i+=1
    if i % 40 == 0:
        print(i)

40
80
120
160
200
240
280
320
360


In [162]:
submission

,id,sing1,sing2,sing3,sing4,sing5,sing6,sing7,sing8
0,754,31,22,0,0,0,0,0,0
1,29,5,5,4,4,0,0,0,0
2,1157,8,61,0,0,0,0,0,0
3,1049,0,0,0,0,0,0,0,0
4,715,8,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
383,243,28,29,0,0,0,0,0,0
384,921,8,5,14,4,0,0,0,0
385,881,8,0,0,0,0,0,0,0
386,503,8,0,0,0,0,0,0,0


In [163]:
submission.to_csv('my_submission.csv', index=False)

In [96]:
ss = pd.read_csv('sample_solution.csv')
ss

,id,sing1,sing2,sing3,sing4,sing5,sing6,sing7,sing8
0,754,0,0,0,0,0,0,0,0
1,29,0,0,0,0,0,0,0,0
2,1157,0,0,0,0,0,0,0,0
3,1049,0,0,0,0,0,0,0,0
4,715,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
383,243,0,0,0,0,0,0,0,0
384,921,0,0,0,0,0,0,0,0
385,881,0,0,0,0,0,0,0,0
386,503,0,0,0,0,0,0,0,0
